## Chapter 12

### Tensorflow

* 강력한 수치 계산용 라이브러리. 특히 대규모 머신러닝에 잘 맞도록 튜닝되어있음.
  * numpy와 비슷한 구조, GPU지원.
  * 분산 컴퓨팅 지원
  * JIT 컴파일러 포함. 속도를 높이고 메모리 사용량을 줄이기 위해 계산을 최적화하고, 이를 위해 계산 그래프를 추출해 최적화하여 효율적으로 실행.
  * 한 환경에서 텐서플로 모델을 훈련하고 다른 환경에서 실행가능.
  * 자동 미분, 고성능 optimizer 제공.
* 위의 핵심 기술을 기반으로 매우 많은 기능 제공
  * <code>tf.keras</code>가 가장 중요
  * 데이터 적재 및 전처리 연산(<code>tf.data, tf.io</code>등), 이미지 처리 연산(<code>tf.image</code>), 신호 처리 연산(<code>tf.signal</code>) 등의 기능을 제공
* 많은 연산이 커널(kernel)이라고 부르는 여러 구현을 가짐.
* 각 커널은 CPU, GPU, TPU(텐서 연산 장치, Tensor processing unit)와 같은 특정 장치에 맞춰 만들어짐.
  * GPU는 계산을 작은 단위로 나눠 여러 GPU thread에서 병렬로 실행하므로 속도를 향상시킴.
  * TPU는 gPU보다 더 빠름.
* 시각화를 위해 TensorBoard, 텐서플로 제품화를 위한 라이브러리 모음인 TFX(https://tensorflow.org/tfx) (데이터 시각화, 전처리, 모델 분석, serving 등 포함), 사전훈련된 신경망을 다운받을 수 있는 텐서플로 허브(https://github.com/tensorflow/models) 등 사용가능.

### Numpy처럼 Tensorflow 사용하기

* Tensor는 한 연산에서 다른 연산으로 흐름.
* Tensor는 <code>ndarray</code>와 비슷. (즉, 일반적으로 다차원 배열)
  * 스칼라 값(42 등)도 가질 수 있음.

In [3]:
# 텐서 생성
# 두 개의 행과 세 개의 열을 가진 실수 행렬

import tensorflow as tf

tf.constant([[1., 2., 3.], [4., 5., 6.]])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [6]:
# 스칼라 생성

tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

* ndarray와 마찬가지로 <code>tf.Tensor</code>는 크기와 데이터 타입(dtype)을 가짐.

In [8]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])

In [9]:
t.shape

TensorShape([2, 3])

In [10]:
t.dtype

tf.float32

In [11]:
# index 참조

t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [12]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

* 모든 종류의 텐서 연산 가능

In [13]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [15]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [17]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

> * t + 10 : tf.add(t, 10)
>   * -, * 등도 지원
> * @ : 행렬 곱셈. tf.matmul()과 동일
* 기본적 수학 연산 (<code>tf.add(), tf.multiply(), tf.square(), tf.exp(), tf.sqrt()</code>등)과 numpy의 대부분의 연산들(<code>tf.reshape(), tf.squeeze(), tf.tile()</code>)을 제공.
  * 단, 일부는 조금씩 다름(<code>tf.reduce_mean()</code> == <code>np.mean()</code> 등)

* keras API에서는 <code>keras.backend</code>에 자체적인 저수준 API를 포함.
  * <code>square(), exp(), sqrt()</code>등. 이들은 상응하는 텐서플로 연산을 호출.
  * 다른 케라스 구현에 적용할 수 있는 코드를 작성하려면 이러한 케라스 함수를 사용해야함.

In [18]:
from tensorflow import keras

K = keras.backend
K.square(K.transpose(t)) + 10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

* ndarray <-> Tensor 간 변환이 가능. 또한, ndarray에 텐서플로 연산을 사용할 수 있고, Tensor에 넘파이 연산을 사용할 수 있음.

In [20]:
import numpy as np

a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [21]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [22]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [23]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

> numpy는 64bit 정밀도를 사용하고 tensorflow는 32bit 정밀도를 사용하므로 numpy 배열로 tensor를 만들려면 <code>dtype=tf.float32</code>로 지정해야함.

* 타입 변환이 가능하나, 성능을 크게 감소시킬 수 있고 타입이 자동으로 변환되면 사용자가 눈치채지 못할 수 있으므로 텐서플로에서는 타입 변환을 자동으로 수행하지 않음.

In [ ]:
tf.constant(2.) + tf.constant(40)

> InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]

In [ ]:
tf.constant(2.) + tf.constant(40., dtype=tf.float64)

> InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2]

In [26]:
# 단, type 변환이 필요한 경우 바꿀 수는 있음. (tf.cast())

t2 = tf.constant(40, dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

* tf.Tensor는 변경이 불가능한 객체. 즉, 일반적인 텐서로는 역전파로 변경해야 하는 신경망의 가중치를 구현할 수 없음.
* 이를 해결하기 위해 <code>tf.Variable</code> 사용.
* tf.Tensor와 비슷하게 동작함. 대신, <code>assign()</code>을 이용해 변수값을 바꿀 수 있음.
  * <code>assign_add(), assign_sub()</code>를 이용해 변수값 증가, 감소도 가능.
  * 또한, 원소의 <code>assign()</code>이나 <code>scatter_update(), scatter_nd_update()</code>로 개별 원소 또는 슬라이스를 수정할 수 있음.

In [29]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])

In [30]:
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [31]:
v.assign(2*v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [32]:
v[0, 1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [33]:
v[:, 2].assign([0., 1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [34]:
v.scatter_nd_update(indices=[[0, 0], [1,2]], updates=[100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

> Keras에서는 <code>add_weight()</code>로 변수 생성을 대신하기 때문에 변수를 직접 만드는 일은 잘 없고, 변수 업데이트도 optimizer가 하기 때문에 수동으로 업데이트하는 일도 거의 없음.